In [384]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [385]:
from fastai.structured import *
from fastai.column_data import *
from fastai.imports import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics
np.set_printoptions(threshold=50, edgeitems=20)

PATH='data/hackerearth/'

In [386]:
train_i=pd.read_csv(f'{PATH}train.csv',low_memory=False)

In [387]:
train_i.head()

,area_assesed,building_id,damage_grade,district_id,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,has_geotechnical_risk_other,has_geotechnical_risk_rock_fall,has_repair_started,vdcmun_id
0,Both,24385bfd2a2,Grade 4,24,0.0,0,0,0,0,0,0,0,0.0,2489
1,Both,405d1bbebbf,Grade 2,44,0.0,0,0,0,0,0,0,0,1.0,4423
2,Both,351d9bc71f6,Grade 1,36,0.0,0,0,0,0,0,0,0,0.0,3650
3,Building removed,2be3a971166,Grade 5,30,0.0,0,0,0,0,0,0,0,0.0,3016
4,Both,34c7d073ea6,Grade 3,36,0.0,0,0,0,0,0,0,0,0.0,3627


In [388]:

train_i.shape

(631761, 14)

In [389]:
test_i=pd.read_csv(f'{PATH}test.csv',low_memory=False)

In [390]:
test_i.head()

,area_assesed,building_id,district_id,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,has_geotechnical_risk_other,has_geotechnical_risk_rock_fall,has_repair_started,vdcmun_id
0,Both,a3380c4f75,7,0.0,0,0,0,0,0,0,0,1.0,701
1,Both,a338a4e653,7,0.0,0,0,0,0,0,0,0,1.0,701
2,Building removed,a338a4e6b7,7,0.0,0,0,0,0,0,0,0,1.0,701
3,Both,a33a6eaa3a,7,0.0,0,0,0,0,0,0,0,1.0,701
4,Building removed,a33b073ff6,7,0.0,0,0,0,0,0,0,0,1.0,701


In [391]:

test_i.shape

(421175, 13)

In [392]:
build_si=pd.read_csv(f'{PATH}Building_Structure.csv',low_memory=False)

In [393]:
build_oi=pd.read_csv(f'{PATH}Building_Ownership_Use.csv',low_memory=False)

In [394]:
build_si.head()
build_si.shape

(1052948, 29)

In [395]:
build_oi.head()
build_oi.shape

(1052948, 17)

In [396]:
build_i=pd.merge(build_si,build_oi,on='building_id')

In [397]:
build_i.head()
build_i.shape

(1052948, 45)

In [398]:
test_f=pd.merge(test_i,build_i,on= 'building_id',how='left')

In [399]:
test_f.shape

(421175, 57)

In [400]:
train_f=pd.merge(train_i,build_i,on= 'building_id',how='left')

In [401]:
train_f.shape

(631761, 58)

In [402]:
train_test= pd.concat([train_f,test_f],join='outer') 
train_test.shape

/home/p_abhijeet666/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


(1052936, 58)

In [403]:
train_cats(train_test)


In [404]:
#train_cats(test_f)

In [405]:
#os.makedirs('tmp', exist_ok=True)
#train_test.to_feather('tmp/bulldozers-raw')

In [406]:
#train_test = pd.read_feather('tmp/bulldozers-raw')

In [407]:
train_test.shape

(1052936, 58)

In [408]:
df, y, nas = proc_df(train_test, 'damage_grade')

In [409]:
#df.shape

In [410]:
#df_test, y_test, nas = proc_df(test_f, 'damage_grade')

In [411]:
#df_test.shape

In [412]:
#preds=m.predict(df_test)

In [413]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 421175  # same as Kaggle's test set size
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(train_test, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape

((631761, 59), (631761,), (421175, 59))

def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

In [414]:
m = RandomForestClassifier(n_jobs=-1)
m.fit(X_train, y_train)
m.score(X_train,y_train)

0.9897112990513818

In [415]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

In [416]:
rmse(m.predict(X_train), y_train)

0.12097636703150816

In [417]:
m.score(X_train, y_train)

0.9897112990513818

In [418]:
preds=m.predict(X_valid)

In [419]:
preds[0:10]

array([3, 4, 4, 2, 4, 2, 1, 4, 2, 3], dtype=int8)

In [420]:
y_train[0:10]

array([3, 1, 0, 4, 2, 4, 1, 4, 0, 2], dtype=int8)

In [421]:
preds2=m.predict(X_train)
preds2[0:10]

array([3, 1, 0, 4, 2, 4, 1, 4, 0, 2], dtype=int8)

In [422]:
np.savez(f'{PATH}regr_resid',X_train['building_id'] ,preds)

In [423]:
data=pd.DataFrame(test_i['building_id'], columns=['building_id'],index=None)
data['damage_grade']=preds
preds.size

421175

In [424]:
test_i['building_id'].size

421175

In [425]:
#data=np.load(f'{PATH}regr_resid.npz')

In [426]:
data.head()

,building_id,damage_grade
0,a3380c4f75,3
1,a338a4e653,4
2,a338a4e6b7,4
3,a33a6eaa3a,2
4,a33b073ff6,4


In [427]:
data.replace(0,'grade 1',inplace=True)
data.replace(1,'grade 2',inplace=True)
data.replace(2,'grade 3',inplace=True)
data.replace(3,'grade 4',inplace=True)
data.replace(4,'grade 5',inplace=True)

In [428]:
data.head()

,building_id,damage_grade
0,a3380c4f75,grade 4
1,a338a4e653,grade 5
2,a338a4e6b7,grade 5
3,a33a6eaa3a,grade 3
4,a33b073ff6,grade 5


In [431]:
data.to_csv('data.csv',index=False)

In [432]:
data.shape

(421175, 2)